# 🤖 Multi-Domain Intelligent Assistant
## Supervisor Agent Architecture with Corrective RAG

**AI 574: Natural Language Processing — Spring 2026**

This notebook demonstrates the complete system:
1. **Foundation** — Load LLM (Gemma 2 9B) and embedding model
2. **Ingestion** — Index documents across 3 domains
3. **CRAG Pipeline** — Self-correcting retrieval-augmented generation
4. **Agents** — Domain-specialized industrial, recipe, and scientific agents
5. **Supervisor** — Intent classification and routing
6. **Evaluation** — Routing accuracy, retrieval quality, LLM-judge

## 0. Setup & Installation

In [ ]:
# Install dependencies
!pip install -q langchain langchain-core langgraph \
    keras keras-hub jax[cuda12] \
    sentence-transformers chromadb \
    PyPDF2 arxiv pydantic

In [ ]:
# Clone or upload project files
# Option A: Upload the project/ folder to Colab
# Option B: Mount Google Drive
# from google.colab import drive
# drive.mount('/content/drive')

import sys
sys.path.insert(0, '/content/project')  # Adjust path as needed

# Set JAX backend for Keras
import os
os.environ['KERAS_BACKEND'] = 'jax'

import logging
logging.basicConfig(level=logging.INFO)

## 1. Foundation Layer — Load Models

In [ ]:
from config.settings import CONFIG, LLMConfig
from foundation.llm_wrapper import KerasHubChatModel
from foundation.embedding_service import EmbeddingService
from foundation.vector_store import VectorStoreService

# Print configuration
print(f"LLM: {CONFIG.llm.preset}")
print(f"Embeddings: {CONFIG.embedding.model_name}")
print(f"Domains: {CONFIG.supervisor.domains}")

In [ ]:
# Load the LLM (this takes ~2-3 minutes on A100)
print("Loading LLM...")
llm = KerasHubChatModel(config=CONFIG.llm)
print("✅ LLM loaded")

In [ ]:
# Load embedding model
print("Loading embedding model...")
embedder = EmbeddingService(config=CONFIG.embedding)
print("✅ Embedding model loaded")

# Quick test
test_vec = embedder.embed_query("PLC fault code")
print(f"Embedding dimension: {len(test_vec)}")

In [ ]:
# Initialize vector store
vector_store = VectorStoreService(embedding_service=embedder)
print("✅ Vector store initialized")
print(vector_store.get_all_stats())

## 2. Data Ingestion — Index Domain Knowledge

In [ ]:
from ingestion.index_builder import IndexBuilder

index_builder = IndexBuilder(vector_store=vector_store)

# ── Industrial Domain ──
# Option A: Load from PDF manuals directory
# index_builder.index_industrial_docs('./data/manuals/')

# Option B: Index sample texts for demo
sample_industrial = [
    """Fault Code F0003 - Siemens S7-1200 PLC: Motor Overtemperature.
    Cause: Motor temperature has exceeded the configured warning threshold.
    Troubleshooting Steps:
    1. Check motor cooling fan operation
    2. Verify ambient temperature is within specifications
    3. Check for mechanical binding or excessive load
    4. Inspect motor winding resistance for signs of degradation
    5. Review VFD output current vs motor nameplate rating
    Resolution: Reduce load or improve cooling. Reset fault after temperature drops.""",

    """PROFINET Configuration Guide - Siemens S7-1200 to S7-1500 Communication.
    Prerequisites: Both PLCs must be on the same subnet.
    Step 1: In TIA Portal, open the network view
    Step 2: Assign IP addresses (e.g., 192.168.0.1 and 192.168.0.2)
    Step 3: Create a PROFINET IO system connection
    Step 4: Configure I/O data exchange modules
    Step 5: Download configurations to both controllers
    Step 6: Verify communication using diagnostic buffer""",

    """Preventive Maintenance Schedule - Allen-Bradley PowerFlex 525 VFD.
    Monthly: Inspect cooling fans, check for dust accumulation
    Quarterly: Verify DC bus voltage, check capacitor health indicators
    Semi-annually: Tighten all power connections, inspect for signs of overheating
    Annually: Full parameter backup, firmware version check, thermal imaging scan
    MTBF: Approximately 28 years under normal operating conditions.""",
]

count = index_builder.index_industrial_texts(sample_industrial, source="demo_manual")
print(f"✅ Indexed {count} industrial chunks")

In [ ]:
# ── Recipe Domain ──
# Option A: Load from Food.com CSV
# index_builder.index_recipes('./data/RAW_recipes.csv', max_rows=50000)

# Option B: Sample data for demo
from ingestion.document_loader import DocumentLoader
from ingestion.chunking_pipeline import ChunkingPipeline

sample_recipes = [
    """Recipe: Classic Chocolate Chip Cookies
    Prep Time: 45 minutes
    Ingredients: 2 1/4 cups flour, 1 tsp baking soda, 1 tsp salt,
    1 cup butter softened, 3/4 cup sugar, 3/4 cup brown sugar,
    2 large eggs, 2 tsp vanilla, 2 cups chocolate chips
    Steps: 1. Preheat oven to 375F. 2. Mix dry ingredients.
    3. Cream butter and sugars. 4. Beat in eggs and vanilla.
    5. Gradually blend in flour mixture. 6. Stir in chips.
    7. Drop rounded tablespoons onto baking sheets.
    8. Bake 9 to 11 minutes until golden brown.
    Egg Substitutes: For egg-free version, use 1/4 cup unsweetened
    applesauce per egg or 1 mashed banana per egg.""",

    """Recipe: Quick Chicken Stir-Fry
    Prep Time: 20 minutes
    Ingredients: 1 lb chicken breast, 2 bell peppers, 1 cup rice,
    3 tbsp soy sauce, 1 tbsp sesame oil, 2 cloves garlic, ginger
    Steps: 1. Cook rice according to package directions.
    2. Slice chicken into strips. 3. Heat oil in wok over high heat.
    4. Stir-fry chicken 5-6 minutes. 5. Add vegetables and garlic.
    6. Add soy sauce and ginger. 7. Serve over rice.
    Nutrition: ~450 calories per serving, 35g protein.""",
]

recipe_docs = DocumentLoader.from_texts(sample_recipes, "recipe", "demo_recipes")
chunker = ChunkingPipeline()
recipe_chunks = chunker.chunk_documents(recipe_docs, domain="recipe")
count = vector_store.add_documents("recipe", recipe_chunks)
print(f"✅ Indexed {count} recipe chunks")

In [ ]:
# ── Scientific Domain ──
# ArXiv papers are fetched on-demand by the scientific agent,
# but we can pre-index some for faster demos
try:
    count = index_builder.index_arxiv_papers(
        "transformer attention mechanism", max_results=5
    )
    print(f"✅ Indexed {count} scientific chunks")
except Exception as e:
    print(f"⚠️ ArXiv fetch failed (network issue?): {e}")
    print("Scientific agent will try on-demand fetching at query time")

In [ ]:
# Check indexing status
print("\n📊 Index Status:")
for stat in vector_store.get_all_stats():
    print(f"  {stat['domain']:>12}: {stat['document_count']} chunks")

## 3. Build & Run the Workflow

In [ ]:
from orchestration.workflow_graph import build_workflow, run_query

# Build the complete LangGraph workflow
workflow = build_workflow(
    llm=llm,
    vector_store=vector_store,
    index_builder=index_builder,
)
print("✅ Workflow compiled")

In [ ]:
# ── Test: Industrial Query ──
result = run_query(workflow, "My S7-1200 PLC is showing fault code F0003")
print(f"Domain: {result['domain']}")
print(f"Confidence: {result['confidence']:.2f}")
print(f"\nResponse:\n{result['response']}")
print(f"\nSources: {result['sources']}")

In [ ]:
# ── Test: Recipe Query ──
result = run_query(workflow, "What can I substitute for eggs in cookies?")
print(f"Domain: {result['domain']}")
print(f"Confidence: {result['confidence']:.2f}")
print(f"\nResponse:\n{result['response']}")

In [ ]:
# ── Test: Scientific Query ──
result = run_query(workflow, "Summarize recent papers on transformer attention")
print(f"Domain: {result['domain']}")
print(f"Confidence: {result['confidence']:.2f}")
print(f"\nResponse:\n{result['response']}")

In [ ]:
# ── Test: Ambiguous Query (should trigger clarification) ──
result = run_query(
    workflow,
    "What temperature should I cook chicken to avoid equipment failure?"
)
print(f"Domain: {result['domain']}")
print(f"Needs clarification: {result['needs_clarification']}")
print(f"\nResponse:\n{result['response']}")

## 4. Evaluation

In [ ]:
from evaluation.metrics import Evaluator, INDUSTRIAL_TEST_QUERIES

evaluator = Evaluator(workflow=workflow, llm=llm)

# Run routing evaluation on a subset
routing_results = evaluator.evaluate_routing(
    test_queries=INDUSTRIAL_TEST_QUERIES[:5]  # Quick test with 5 queries
)

print(f"Routing Accuracy: {routing_results['overall_accuracy']:.2%}")
print(f"Per-Domain: {routing_results['per_domain_accuracy']}")
print(f"\nDetailed Results:")
for r in routing_results['results']:
    status = '✅' if r['correct'] else '❌'
    print(f"  {status} [{r['actual']:>12}] {r['query'][:60]}...")

In [ ]:
# LLM-Judge quality assessment
sample_result = run_query(workflow, "How to reset a PowerFlex 525 drive?")
judge_score = evaluator.evaluate_llm_judge(
    query="How to reset a PowerFlex 525 drive?",
    response=sample_result['response'],
    domain=sample_result['domain'],
)
print("LLM-Judge Scores:")
for k, v in judge_score.items():
    print(f"  {k}: {v}")

## 5. Interactive Demo

In [ ]:
# Interactive query loop
print("🤖 Multi-Domain Assistant Ready!")
print("Domains: Industrial | Recipe | Scientific")
print("Type 'quit' to exit\n")

while True:
    query = input("You: ")
    if query.lower() in ('quit', 'exit', 'q'):
        print("Goodbye!")
        break

    result = run_query(workflow, query)
    print(f"\n[Routed to: {result['domain']} | Confidence: {result['confidence']:.2f}]")
    print(f"Assistant: {result['response']}\n")